# sample points from ECM in each subregion.
Sampling points from ECM is a three step process:  
(i) For each pixel in the image with pixel value `p`, sample a point from Binomial(p^2).  
(ii) Remove "island" points: points that have less than M points in its neighborhood.  
(iii) Downsample

In [ ]:
include("src/ECM_TDA.jl")
using .ECM_TDA

using Images
using CSV
using Plots
using JLD2
using Random
using StatsBase
using Distributions
using DataFrames


In [ ]:
gr()
c_ECM = "#259ea1"

# 1. Illustration of sampling process in example image

Load example sub-image

In [ ]:
Examples = [("145", "211", 3), ("001", "374", 2), ("021", "110", 1), ("049", "134", 1), ("124", "104", 1)]

In [ ]:
# open image
i = 1
LTX = Examples[i][1]
Da = Examples[i][2]
idx = Examples[i][3]

image_path = "data/4000x4000/subregion_ECM/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".tif"

img = Array(Images.load(image_path));

In [ ]:
resampled, points, sampled, island_idx = sample_ECM_points(img, vicinity = 100, n_points = 5)

Plot all steps of the sampling process

In [ ]:
gr()
p1 = plot(Gray.(img), frame = :box, ticks = [])
p2 = plot(Gray.(1 .- sampled), frame = :box, ticks = [])
p3 = plot(Gray.(1 .- sampled), frame = :box, ticks = [])
scatter!(p3, points[island_idx,1], points[island_idx,2], yflip = :true, c = "#ff6b6b", label = "", frame = :box, ticks = [], markershape = :xcross, markersize = 3, markerstrokewidth = 2)
p4 = scatter(resampled[:,1], resampled[:,2], yflip = :true, c = c_ECM, label = "", frame = :box, ticks = [])
p = plot(p1, p2, p3, p4, layout = grid(1,4), size = (1200, 300), background_color=:transparent, foreground_color=:black)
plot(p)
savefig("figures/sampling_process.svg")

# 2. Sample points from all ECM images
* Determine the number of points to sample from each ECM image  
    * based on histogram of mean ECM pixel values
* For each ECM subregion, sample points
    * save the figure of the final points and removed islands (for reference)
    * save locations of final points for analysis

## 2(a) Determine the number of points to sample from ECM image.

Load all ECM images

In [ ]:
function get_ECM_mean_pixels(directory)
    files =  [item for item in walkdir(directory)][1][3]
    ECM_mean_pixels = []
    for filename in files
        img = Array(Images.load(directory * filename))
        push!(ECM_mean_pixels, mean(Float64.(img)))
    end
    return ECM_mean_pixels
end

In [ ]:
ECM_directory1 = "data/4000x4000/subregion_ECM/"
ECM_directory2 = "data/4000x4000_201222/subregion_ECM_green/"
ECM_directory3 = "data/4000x4000_201222/subregion_ECM_purple/"
ECM_pixels1 = get_ECM_mean_pixels(ECM_directory1)
ECM_pixels2 = get_ECM_mean_pixels(ECM_directory2)
ECM_pixels3 = get_ECM_mean_pixels(ECM_directory3);

In [ ]:
# invert ECM values so that mean pixel value close to 1 has high ECM content
ECM_pixels = vcat(ECM_pixels1, ECM_pixels2, ECM_pixels3)
ECM_mean_inv =  1 .- ECM_pixels
low = percentile(ECM_mean_inv, 1)
high = percentile(ECM_mean_inv, 99)

println("Mean pixel value at 1%: ", low)
println("Mean pixel value at 99%: ", high)

In [ ]:
# plot histogram
p = histogram(ECM_mean_inv, label = "", 
    xlabel = "ECM mean pixel value", ylabel = "frequency",
    guidefontsize = 15, xtickfontsize = 12, ytickfontsize = 12, legendfontsize = 12,
    color = "grey77",
    size = (500, 350))
vline!(p, [low, low], label = "1%", linewidth = 5, color = "coral2")
vline!(p, [high, high], label = "99%", linewidth = 5, color = "lightseagreen")
savefig("data/4000x4000_combined/ECM_pixel_histogram.pdf")
plot(p)

In [ ]:
# plot the number of points to be sampled as a function of ECM mean pixel value
min_sample = 100 
max_sample = 5000

x = range(0, 1, length = 100)
y = n_ECM_samples.(x, low, high, min_sample, max_sample)
p = plot(x, y, label = "", 
        xlabel = "ECM mean pixel value", ylabel ="# samples from ECM",
        color = "grey",
        linewidth = 5,
        legend = :bottomright,
        guidefontsize = 15, xtickfontsize = 12, ytickfontsize = 12, legendfontsize = 12,
        size = (500, 350),
        )
vline!(p, [low, low], label = "1%", linewidth = 5, color = "coral2")
vline!(p, [high, high], label = "99%", linewidth = 5, color = "lightseagreen")
savefig("data/4000x4000_combined/n_ECM_function.pdf")
plot(p)

## 2(b) Sample points from all ECM images

Sample from original ROI

In [ ]:
function sample_points_from_ECM_directory(ECM_directory, min_sample, max_sample, low, high, c_ECM;
                                         sample_plot_directory = "data/4000x4000_combined/ECM_sampled/sampled_points/",
                                         sample_CSV_directory = "data/4000x4000_combined/ECM_sampled/points_CSV/")
    files = [item for item in walkdir(ECM_directory)][1][3]
    for filename in files
        img = Array(Images.load(ECM_directory * filename))
        figure_file = split(filename,".")[1] * ".pdf"

        # compute (inverted) mean pixel value of image
        img_mean_inv = 1- mean(Float64.(img))

        # compute number of points to sample
        n_sample = Int64(round(ECM_TDA.n_ECM_samples(img_mean_inv, low, high, min_sample, max_sample)))

        # sample points
        resampled, points, sampled, island_idx = sample_ECM_points(img, vicinity = 100, n_points = 5, n_samples = n_sample)

        # plot the results 
        p = scatter(resampled[:,1], resampled[:,2], yflip = :true, c = c_ECM, label = "", frame = :box, ticks = [], size = (500, 500))
        savefig(sample_plot_directory * figure_file)

        # save sampled points to CSV
        csv_file = split(filename, ".")[1] * ".csv"
        df = DataFrame(resampled, [:x, :y])
        CSV.write(sample_CSV_directory * csv_file, df)
    end
end



In [ ]:
sample_points_from_ECM_directory(ECM_directory1, min_sample, max_sample, low, high, c_ECM)
sample_points_from_ECM_directory(ECM_directory2, min_sample, max_sample, low, high, c_ECM)
sample_points_from_ECM_directory(ECM_directory3, min_sample, max_sample, low, high, c_ECM)

# 3. Check alignment

Example image from original ROI

In [ ]:
LTX = "001"
Da = "232"
idx = 1

# load ECM image
image_path = "data/4000x4000/subregion_ECM/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".tif"
img = Array(Images.load(image_path));

# load sampled ECM
ecm_points_path = "data/4000x4000_combined/ECM_sampled/points_CSV/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".csv" 
ECM_points = CSV.read(ecm_points_path);

In [ ]:
# plot ECM and sampled ECM points
p1 = plot(Gray.(img), frame = :box, ticks = [], size = (300, 300), background_color=:transparent, foreground_color=:black)
p2 = scatter(ECM_points.x, ECM_points.y, yflip = :true, c = c_ECM, label = "", frame = :box, ticks = [], markersize = 2, background_color=:transparent, foreground_color=:black)
plot(p1, p2, size = (600, 300))

Example image from new ROI

In [ ]:
examples = [("092", "523", 4), ("049", "134", 3), ("097", "113", 3)]

In [ ]:
LTX = "092"
Da = "523"
idx = 4

# load ECM image
image_path = "data/4000x4000_combined/subregion_ECM/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".tif"
img = Array(Images.load(image_path));

# load sampled ECM
ecm_points_path = "data/4000x4000_combined/ECM_sampled/points_CSV/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".csv" 
ECM_points = CSV.read(ecm_points_path);

In [ ]:
# plot ECM and sampled ECM points
p1 = plot(Gray.(img), frame = :box, ticks = [], size = (300, 300), background_color=:transparent, foreground_color=:black)
p2 = scatter(ECM_points.x, ECM_points.y, yflip = :true, c = c_ECM, label = "", frame = :box, ticks = [], markersize = 2, background_color=:transparent, foreground_color=:black)
plot(p1, p2, size = (600, 300))

# 4. (for figure) save example images with low and high mean ECM pixel 

In [ ]:
# find image with mean pixel value close to "low"
idx_low = index_closest_to_x(ECM_mean_inv, low)

# find image with mean pixel value close to "high"
high = percentile(ECM_mean_inv, 99.5)
idx_high = index_closest_to_x(ECM_mean_inv, high)

# find image with mean pixel value around median
med = median(ECM_mean_inv)
idx_median = index_closest_to_x(ECM_mean_inv, med);

In [ ]:
println("low: ", files[idx_low])
println("median: ", files[idx_median])
println("high: ", files[idx_high])

Copy ECM image files for three examples

In [ ]:
# plot low
examples = [("092", "523", 4), ("049", "134", 3), ("097", "113", 3)]
for ex in examples
    LTX = ex[1]
    Da = ex[2]
    idx = ex[3]

    # load ECM image
    image_path = "data/4000x4000/subregion_ECM/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".tif"
    img = Array(Images.load(image_path));

    # save copy of ECM image (with reduced size)
    filename = "data/4000x4000/subregion_ECM/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".tif" 
    copied_filename = "data/4000x4000/ECM_sampled/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".tif" 
    cp(filename, copied_filename, force = true)
    # reduce size
    run(`convert $copied_filename -resize 500x500 $copied_filename`)
end

In [ ]:
# print info
println("low")
println("ECM pixel value (inverted): ", ECM_mean_inv[idx_low])
println("number of points sampled: ", n_ECM_samples(ECM_mean_inv[idx_low], low, high, min_sample, max_sample))

println("median")
println("ECM pixel value (inverted): ", ECM_mean_inv[idx_median])
println("number of points sampled: ", n_ECM_samples(ECM_mean_inv[idx_median], low, high, min_sample, max_sample))

println("high")
println("ECM pixel value (inverted): ", ECM_mean_inv[idx_high])
println("number of points sampled: ", n_ECM_samples(ECM_mean_inv[idx_high], low, high, min_sample, max_sample))

In [ ]:
LTX = "092"
Da = "523"
idx = 4

# load ECM image
image_path = "data/4000x4000_combined/subregion_ECM/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".tif"
img = Array(Images.load(image_path));

# load sampled ECM
ecm_points_path = "data/4000x4000_combined/ECM_sampled/points_CSV/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".csv" 
ECM_points = CSV.read(ecm_points_path);

p = scatter(ECM_points.x, ECM_points.y, yflip = :true, c = c_ECM, label = "", frame = :box, ticks = [], markersize = 2, background_color=:transparent, foreground_color=:black, size = (300, 300))
savefig("figures/ECM_sample1.svg")

In [ ]:
LTX = "049"
Da = "134"
idx = 3

# load ECM image
image_path = "data/4000x4000_combined/subregion_ECM/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".tif"
img = Array(Images.load(image_path));

# load sampled ECM
ecm_points_path = "data/4000x4000_combined/ECM_sampled/points_CSV/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".csv" 
ECM_points = CSV.read(ecm_points_path);

p = scatter(ECM_points.x, ECM_points.y, yflip = :true, c = c_ECM, label = "", frame = :box, ticks = [], markersize = 2, background_color=:transparent, foreground_color=:black, size = (300, 300))
savefig("figures/ECM_sample2.svg")

In [ ]:
LTX = "097"
Da = "113"
idx = 3

# load ECM image
image_path = "data/4000x4000_combined/subregion_ECM/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".tif"
img = Array(Images.load(image_path));

# load sampled ECM
ecm_points_path = "data/4000x4000_combined/ECM_sampled/points_CSV/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".csv" 
ECM_points = CSV.read(ecm_points_path);

p = scatter(ECM_points.x, ECM_points.y, yflip = :true, c = c_ECM, label = "", frame = :box, ticks = [], markersize = 2, background_color=:transparent, foreground_color=:black, size = (300, 300))
savefig("figures/ECM_sample3.svg")